<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2021_1222onomatopea_bert_fine_turing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- filename: 2021_1222onomatopea_bert_fine_tuing.ipynb
- memo: 2021年12月25日現在，

transformers は M1 Mac では動作しない。Intel Mac such as pasiphae では動作する。

In [ ]:
import os
import sys
import numpy as np
import unicodedata
from termcolor import colored

# 本ファイルを Google Colaboratory 上で実行する場合に，必要となるライブラリをインストールする
import platform
isColab = platform.system() == 'Linux'
if isColab:
    !pip install transformers > /dev/null 2>&1 

    # MeCab, fugashi, ipadic のインストール
    !apt install aptitude swig > /dev/null 2>&1
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y > /dev/null 2>&1
    !pip install mecab-python3 > /dev/null 2>&1
    !git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 2>&1
    !echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a > /dev/null 2>&1
    
    import subprocess
    cmd='echo `mecab-config --dicdir`\"/mecab-ipadic-neologd\"'
    path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                                     shell=True).communicate()[0]).decode('utf-8')

    !pip install 'fugashi[unidic]' > /dev/null 2>&1
    !python -m unidic download > /dev/null 2>&1
    !pip install ipadic > /dev/null 2>&1

In [ ]:
# 近藤先生 (2021年12月22日） から送っていただいた，オノマトペ文章データを読み込む
import jaconv
data_dir = '/Users/asakawa/study/2021kondo_project'
original = {}
n = 0
with open(os.path.join(data_dir,'original.csv'), 'r', encoding='utf8') as f:
    s = f.read()
    for s_ in s.split('\n'):
        if n == 0:
            n += 1
            continue
        idx, sent = s_.split(',')
        
        # Mac と Windows との unicode 符号化の差分を吸収する
        # jaconv.normalize は内部で unicodedata.normalize('NFKC') を呼び出しているので
        # 差異 between Mac and Windows を吸収できる
        sent = ''.join(jaconv.normalize(x) for x in sent)
        original[int(idx)] = sent

print(f'{len(original)} has been read')

2469 has been read


In [ ]:
# 2021/Jan 近藤先生からいただいたオノマトペ辞典のデータの読み込み
import pandas as pd
import jaconv

#ひとつ下の '日本語オノマトペ辞典4500より.xls' は著作権の問題があり，公にできません。
# そのため Google Colab での解法，ローカルファイルよりアップロードする
#from google.colab import files
#uploaded = files.upload()  # ここで `日本語オノマトペ辞典4500より.xls` を指定してアップロードする
data_dir = '/Users/asakawa/study/2021ccap/notebooks'
#onomatopea_excel = '日本語オノマトペ辞典4500より.xlsx'  # オリジナルファイル名，次行は勝手に rename したファイル名
onomatopea_excel = '2021-0325日本語オノマトペ辞典4500より.xls'
onmtp2761 = pd.read_excel(os.path.join(data_dir, onomatopea_excel), sheet_name='2761語')

#すべてカタカナ表記にしてデータとして利用する場合
#`日本語オノマトペ辞典4500` はすべてひらがな表記だが，一般にオノマトペはカタカナ表記されることが多いはず
#onomatopea = list(sorted(set([jaconv.hira2kata(o) for o in onmtp2761['オノマトペ']])))

# Mac と Windows の表記の相違を吸収
onomatopea = list(sorted(set([jaconv.normalize(o) for o in onmtp2761['オノマトペ']])))

print(f'データファイル名: {os.path.join(data_dir, onomatopea_excel)}\n',
      f'オノマトペ単語総数: len(onomatopea):{len(onomatopea)}')

データファイル名: /Users/asakawa/study/2021ccap/notebooks/2021-0325日本語オノマトペ辞典4500より.xls
 オノマトペ単語総数: len(onomatopea):1741


In [ ]:
%%time
import torch
from transformers import BertConfig
from transformers import BertModel
from transformers import BertForPreTraining
from transformers import BertJapaneseTokenizer
from transformers import BertForMaskedLM

model_ja_name = 'cl-tohoku/bert-base-japanese' 
model = BertModel.from_pretrained(model_ja_name)
config = BertConfig.from_pretrained(model_ja_name)

# トークナイザ の修正
tknz1 = BertJapaneseTokenizer.from_pretrained(model_ja_name) # BPE (or sentencepiece) による下位単語分割あり
#siz = len(tknz1)
#siz = len(onomatopea)

#tknz1.ids_to_tokens.update({i+len(onomatopea):o for i,o in enumerate(onomatopea)})
#tknz1.vocab.update({i+len(onomatopea):o for i,o in enumerate(onomatopea)})
tknz1.vocab.update(tknz1.added_tokens_decoder)
tknz1.ids_to_tokens.update(tknz1.added_tokens_decoder)
tknz1.add_tokens(onomatopea)
model.resize_token_embeddings(len(tknz1))

print(len(tknz1), len(tknz1.vocab), tknz1.vocab_size)
print(tknz1.convert_tokens_to_ids(onomatopea[-10:]))
print(tknz1.convert_ids_to_tokens(tknz1.convert_tokens_to_ids(onomatopea[-10:])))

In [ ]:
w = 'わやわや'
print(w in tknz1.vocab)
print(tknz1.tokenize(w))
id_ = tknz1(w)['input_ids'][1]
print(tknz1.convert_ids_to_tokens(id_))

In [ ]:
print(len(tknz1), len(tknz1.vocab), tknz1.vocab_size)
print(tknz1.convert_tokens_to_ids(onomatopea[-10:]))
print(tknz1.convert_ids_to_tokens(tknz1.convert_tokens_to_ids(onomatopea[-10:])))

#tknz1.add_tokens(onomatopea)

print(len(tknz1), len(tknz1.vocab), tknz1.vocab_size)
print(tknz1.convert_tokens_to_ids(onomatopea[-10:]))
print(tknz1.convert_ids_to_tokens(tknz1.convert_tokens_to_ids(onomatopea[-10:])))


In [ ]:
tknz2 = BertJapaneseTokenizer.from_pretrained(model_ja_name)

In [ ]:
print(tknz1.tokenize('雨がしとしとと降る'))
print(tknz2.tokenize('雨がしとしとと降る'))
#help(tknz1.add_tokens)
#tknz1.add_tokens(onomatopea)

In [ ]:
s = 'しとしと'
print(s in onomatopea)
print(tknz1.tokenize(s))
print(tknz1(s))
print(tknz1.convert_ids_to_tokens(tknz1(s)['input_ids'][1]))

#tknz1.vocab.update(tknz1.added_tokens_decoder)
#tknz1.ids_to_tokens.update(tknz1.added_tokens_decoder)
print(tknz1.convert_ids_to_tokens(33710))
print(tknz1.convert_ids_to_tokens(32000))

#list(tknz1.vocab)[-3:]
#tknz1.ids_to_tokens[32000]
tknz1.convert_ids_to_tokens(32000)

print(original[3])

In [ ]:
import re
# ランダムサンプリングしてデータを印字して確認
for _ in range(5):
    N = np.random.randint(low=0, high=len(original))
    sent0 = original[N]
    sent1 = re.sub(r'\(.*\)','',original[N]) # original に含まれる `(と)` のような表現を削除する
    
    print(colored(sent0, attrs=['bold']))  # 送っていただいた元の文
    print(colored('分かち書き','blue'), tknz1.tokenize(sent0)) # その分かち書き
    print(colored('ID 化', 'blue'), tknz1.encode(sent0))   # 分かち書き結果の単語 ID 化

    if sent0 != sent1:
        print(colored(sent1, attrs=['bold']))   # (と) を取り去った文
        print(colored('分かち書き','blue'), tknz1.tokenize(sent1)) # その分かち書き
        print(colored('ID 化', 'blue'), tknz1.encode(sent1))   # 分かち書き結果の単語 ID 化

# MeCab で単語分割が行われて、MeCab が単語として認識しても、その単語が語鎮リスト vocab.txt に登録されていない場合は
# subword である WordPiece が起動され、その単語が適当に分割されます。そのように分割された単語には '##' が単語の前に付与されます。
# また、未知語の場合もWordPieceが起動され、同様に分割されます。
print(colored(f'\ntknz.all_special_ids:{tknz1.all_special_ids}',attrs=['bold']))  #  [1, 3, 0, 2, 4]
print(colored(f'tknz.all_special_tokens:{tknz1.all_special_tokens}', attrs=['bold']))  #  ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

美しいものなどにうっとり(と)晄惚とする


NameError: name 'tknz1' is not defined

In [ ]:
#len(tknz1.vocab)

#print(dir(tknz1))  # トークナイザの内部を調べてみましょう

# # トークナイザの属性値を表示して理解を深める
# for k in ['sep_token', 'sep_token_id', 'slow_tokenizer_class', 
#           'unk_token', 'unk_token_id', 'verbose', #'vocab', 
#           'vocab_files_names', 'vocab_size', 'word_tokenizer', 'word_tokenizer_type',
#           'special_tokens_map', 'special_tokens_map_extended', 
#           'subword_toke0nizer', 'subword_tokenizer_type',
#           'all_special_ids', 'all_special_tokens', 'unk_token', 'unk_token_id', 
#           'special_tokens_map', 'special_tokens_map_extended', 
#           'subword_tokenizer', 'subword_tokenizer_type', 
#           'tokenize',
#           'max_model_input_sizes', 'mecab_kwargs', 'model_input_names', 'model_max_length']:
#     print(colored(f'{k}','blue', attrs=['bold']), colored(f'{getattr(tknz1,k)}','green', attrs=['bold']))

In [ ]:
tknz1.save_vocabulary('vocab_saved.txt')  # 後に利用可能なように，語彙辞書をテキストファイルとして書き出す

# 結果の確認
!head vocab_saved.txt
!tail vocab_saved.txt
#help(tknz.save_vocabulary)

In [ ]:
# for w in ['ひゅー', 'ぴゅー', 'すっかり']:
#     print(w, w in onomatopea)
    
# print(len(original))    

In [ ]:
#for idx in original: 
for idx in range(8): 
    sent = original[idx]
    print(f'sent:{sent}', end="---")
    print('/'.join(tknz1.tokenize(sent)))
    input_ids = tknz1(sent)['input_ids']
    #print(tknz1.ids_to_tokens[input_ids])
    for idx in input_ids:
        print(f'{tknz1.convert_ids_to_tokens(idx)}', end="/")
    print()

In [ ]:
# オノマトペにマッチする部分を検索し，[MASK] で置き換える
masked_onmtp = {}

#for N in np.random.randint(len(original), size=100):
for N in range(0,4):
    sent = original[N] # orginal.csv の内容を 1 行取り出す
    print(f'{N:5,d}', end=" ")
    #print(tknz1.tokenize(sent))
    #print(tknz1(sent)['input_ids'])
    
    for w in tknz1.tokenize(sent):
        color = 'red' if w in onomatopea else 'grey'
        print(colored(w,color), end='/')
    print('\t-->original', colored(sent, 'cyan'))
                
    for i, o in enumerate(onomatopea):
        if sent.find(o) != -1:
            target_id = i
            o_ = o  # 最後にマッチしたオノマトペだけ取り出す
            
    if o_ not in masked_onmtp:
        masked_onmtp[o_] = [sent.replace(o_,'[MASK]')]
    else:
        masked_onmtp[o_].append(sent.replace(o_,'[MASK]'))

#   3 すっ/かり/日/が/とっぷり/(/と/)/暮れ/##る/    from すっかり日がとっぷり(と)暮れる
# 633 泥/##酔/し/て/へ/##べ/##れ/##け/(/と/)/視点/や/口/##調/が/乱れ/##る/    from 泥酔してへべれけ(と)視点や口調が乱れる
# 690 気/どっ/たり/おほん/(/と/)/い/ばっ/たり/する/    from 気どったりおほん(と)いばったりする    #if N > 2500: break
# 924 こちん/と/[UNK]/に/さわ/##る/    from こちんと癪にさわる
#1498 しっかり/と/べたん/と/くっ/つく/    from しっかりとべたんとくっつく
#1829 もの/が/ぎ/##っと/一/瞬/##き/し/##む/    from ものがぎっと一瞬きしむ
#1830 かた/##い/もの/が/ぎ/##し/り/ぎ/##し/り/(/と/)/何/度/も/こ/##す/##れる/    from かたいものがぎしりぎしり(と)何度もこすれる
#1917 金属/を/じゃ/ん/と/たた/##く/    from 金属をじゃんとたたく
#2056 もの/が/他/の/もの/に/ぼ/こ/つ/と/打ち/当たる/    from ものが他のものにぼこつと打ち当たる
#2402 すば/##や/くす/##い/と/動い/たり/変化/する/    from すばやくすいと動いたり変化する

In [ ]:
# オノマトペにマッチする部分を検索し，[MASK] で置き換える

with open('2022_0119ono_test.txt','w') as fo:
    for N in range(len(original)):
        sent = original[N] # orginal.csv の内容を 1 行取り出す
        fo.write(f'{N:5,d} ')
        for w in tknz1.tokenize(sent):
            fo.write(w+'/')
        fo.write(f'\t-->original {sent}\n')

In [ ]:
!tail 2022_0119ono_test.txt

In [ ]:
# 632 酔っ/て/べろんべろん/(/と/)/体/が/まとも/に/動か/せ/ず/    from 酔ってべろんべろん(と)体がまともに動かせず
# 690 気/どっ/たり/おほん/(/と/)/い/ばっ/たり/する/    from 気どったりおほん(と)いばったりする

for w in ['かん高い', 'かん高く', '忙しく', 'じゃんが','へべれけ','ひゅー', 'ぴゅー', 'すっかり']:
    print(w, w in onomatopea)
    
print(len(original))    

In [ ]:
# from transformers import BertForMaskedLM
# masked_lm = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese')

# tknz.add_tokens(onomatopea)
# #print(len(tknz.vocab), len(onomatopea))
# n_added = tknz.add_tokens(onomatopea)    # 東北大学 BERT に登録されていないオノマトペを加える。
# n_added_ = tknz_.add_tokens(onomatopea)  # 東北大学 BERT に登録されていないオノマトペを加える。
# tknz.ids_to_tokens.update()
# tknz_.ids_to_tokens.update()
# masked_lm.resize_token_embeddings(len(tknz))

# #print(tknz.vocab_size)  # vocab_size は変わらないのか
# #help(tknz.get_vocab)
# #print(tknz.get_added_vocab())
# print(len(tknz), len(tknz.vocab), tknz.vocab_size)

# print(tknz.convert_tokens_to_ids(onomatopea[-10:]))
# print(tknz.convert_ids_to_tokens(tknz.convert_tokens_to_ids(onomatopea[-13:])))

# print(tknz_.convert_tokens_to_ids(onomatopea[-10:]))
# print(tknz_.convert_ids_to_tokens(tknz.convert_tokens_to_ids(onomatopea[-13:])))

In [ ]:
from transformers import BertForMaskedLM
masked_lm1 = BertForMaskedLM.from_pretrained(model_ja_name)
masked_lm2 = BertForMaskedLM.from_pretrained(model_ja_name)

tknz1.add_tokens(onomatopea)
n_added = tknz1.add_tokens(onomatopea)    # 東北大学 BERT に登録されていないオノマトペを加える。
tknz1.ids_to_tokens.update()
masked_lm1.resize_token_embeddings(len(tknz1))

tknz2.add_tokens(onomatopea)
n_added_ = tknz2.add_tokens(onomatopea)  # 東北大学 BERT に登録されていないオノマトペを加える。
tknz2.ids_to_tokens.update()
masked_lm2.resize_token_embeddings(len(tknz2))

n_limit = 2
for i, s in enumerate(masked_onmtp):
    print(colored(f'{i} {s}',attrs=['bold']), masked_onmtp[s])
    for s_ in masked_onmtp[s]:
        print(tknz1.tokenize(s_), end="")
        print(colored(tknz1(s_)['input_ids'], 'green'), end="") # with sentencepiece
        x = torch.LongTensor(tknz1(s_)['input_ids']).unsqueeze(0)
        a = masked_lm1(x)
        print(a[0].shape)

        print(tknz2.tokenize(s_), end="")
        print(colored(tknz2(s_)['input_ids'], 'cyan'), end="") # without sentencepiece
        x = torch.LongTensor(tknz2(s_)['input_ids']).unsqueeze(0)
        a = masked_lm2(x)
        print(a[0].shape)

        idxs = tknz2(s_)['input_ids']
        maskpos = idxs.index(tknz2.mask_token_id)
        top_n = 5
        b = torch.topk(a.to_tuple()[0][0][maskpos], k=top_n)
        b_idx = b[1].detach().numpy()
        b_wrd = list(tknz2.convert_ids_to_tokens(b_idx))
        for k in range(top_n):
            for j, idx in enumerate(idxs):
                if j != maskpos:
                    print(colored(tknz2.convert_ids_to_tokens(idx),'blue'), end=" ")
                else:
                    print(colored(b_wrd[k],'red', attrs=['bold']),end=" ")
                    
            print()
    if i > n_limit:
        break

In [ ]:
def print_message(a, s_, tokenizer=tknz1, top_n=5):
    idxs = tokenizer(s_)['input_ids']
    maskpos = idxs.index(tokenizer.mask_token_id)
    b = torch.topk(a.to_tuple()[0][0][maskpos], k=top_n)
    b_idx = b[1].detach().numpy()
    b_wrd = list(tokenizer.convert_ids_to_tokens(b_idx))
    for k in range(top_n):
        print(k, end=":")
        for j, idx in enumerate(idxs):
            if j != maskpos:
                print(colored(tokenizer.convert_ids_to_tokens(idx),'grey'), end=" ")
            else:
                print(colored(b_wrd[k],'red', attrs=['bold']),end=" ")
        print()
    
#for i, s in enumerate(masked_onmtp):
for i in range(5):
    N = np.random.choice(len(masked_onmtp))
    N = 49
    N = 4
    ono = list(masked_onmtp.keys())[N]
    s = masked_onmtp[ono]
    print(colored(f'{i} N:{N} オノマトペ:{ono} {s}', 'blue', attrs=['bold']))
    for s_ in s:
        print(tknz1.tokenize(s_), end="")
        print(colored(tknz1(s_)['input_ids'], 'green'), end="") # with sentencepiece
        x = torch.LongTensor(tknz1(s_)['input_ids']).unsqueeze(0)
        a = masked_lm1(x)
        print(a[0].shape)
        
        print('=' * 7, 'with wordpice')
        print_message(a, s_, tokenizer=tknz1)
        print('+' * 7, 'withoout wordpeice')
        print_message(a, s_, tokenizer=tknz2)


In [ ]:
print(tknz1.tokenize('日が射したり光が[MASK]とともる'))
print(tknz2.tokenize('日が射したり光が[MASK]とともる'))

In [ ]:
tknz1.ids_to_tokens[1]

## 5.7 I BertForMaskedLM の利用 (新納2001, p.128)

BERT モデルはMasked Language Model によっても学習が行われているので、BERT のモデル自体に MASK された単語を推定する機構が含まれています。
MASK された単語を推定するには、モデルからその機構の部分を取り出さなければなりません。
これを行うのが ``BertForMaskedLM`` です。
例文「私は犬が好き。」の「犬」の部分を MASK した以下の単語列に対して、MASK の単語を推定してみます。

> 例文： 私は [MASK] が好き。

まずこの単語列をid 列に変換します(※5)

```python
ids = tknz.encode("私は[MASK]が好き。＂）
ids
[ 2, 1325, 9, 4, 14, 3596, 8, 3]
```

また、以下により [MASK] の位置を確認しておきます。

```python
mskpos = ids.index(tknz.mask_token_id)
mskpos
# 3
```

BERT モデルからの Masked Language Model の取り出しは以下のように行います。

```python
from transformers import BertForMaskedLM
model = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese')
```


次にモデルに単語のid 列を与えると、各単語の位置に現れる単語の分布が得られます。
```python
x = torch.LongTensor(ids).unsqueeze(O)
a = model(x)
```

上記に示したモデルからの出力 `a` は要素が 1つのタプルです。
`a[O]` の形状は以下のとおりです。

> ［ バッチサイズ， 単語列の長さ， 登録単語の数］

この例の場合、1 データだけなのでバッチサイズは 1、単語列の長さは 8 です。
そして登録単語の数はそのモデルの持つ登録単語数です。
この登録単語数は `tknz.vocab_size` から参照できます。
このモデルの場合は `32000` になっています。

```python
a[0].shape  # torch.Size([1, 8, 32000])
tknz.vocab_size # 32000
```

<font size="+1" color="green"> 新納本(2021) 近藤先生から自炊版をもらった pdf では，Bert の出力が 3 連 tuple と記載されている (p.124) が実際は違う
実際には 出力を .to_tuple() で tuple に変換しないといけない。</font>


この例の場合、MASK の位置は `mskpos` だったので、たとえば k=100 番目の登録単語が MASK の位置に現れる程度（確率）は以下のコードで得られます。
```python
k = 100
a[O][O][mskpos][k]
# tensor(-5.5299, grad_fn=<SelectBackward>)
```
変数 `k` を 0 から 31999 まで動かして最も大きな値を持つ $\hat{k}$ を求めれば、$\hat{k}$ 番目の登録単語が MASK の位置に最も高い確率で現れる単語と推定できます。
このようにベタに調べるよりも、torch には `topk` という便利なメソッドがあります。
これはベクトルの要素の中からその値の高いものを上から順に K 個取り出すものです。
以下のように使います。

```ptyhon
b = torch.topk(a[0][0[mskpos],k=5)
b[0]  # 上位 5 つの値
tensor([8.5864, 8.0724, 7.6974, 7.6480, 7.5863] ,...)
b[1］ ＃ 上位 5 つの index
tensor([1301, 1201, 705, 6968, 450])
```

In [ ]:
idxs = tknz.encode('私は[MASK]が好き。')
print(f'idxs :{idxs}')
maskpos = idxs.index(tknz.mask_token_id)
print(f'[MASK] の位置:{maskpos}')

from transformers import BertForMaskedLM
masked_lm = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese')

x = torch.LongTensor(idxs).unsqueeze(0)
a = masked_lm(x)

print(a.to_tuple()[0].shape)  # torch.Size([1, 8, 32000])
print(tknz.vocab_size) # 32000

b = torch.topk(a.to_tuple()[0][0][maskpos],k=5)
print(b[0])     # 上位 5 つの値
# tensor([8.5864, 8.0724, 7.6974, 7.6480, 7.5863] ,...)

print(b[1])     # 上位 5 つの index
# tensor([1301, 1201, 705, 6968, 450])

for idx in b[1].detach().numpy():
    print('idx:', colored(f'{idx}', 'red' ,attrs=['bold']), 'トークン:', colored(f'{tknz.ids_to_tokens[idx]}','red', attrs=['bold']))

for idx in b[1].detach().numpy():
    print('idx:', colored(f'{idx}', 'red' ,attrs=['bold']), 'トークン:', colored(f'{tknz.ids_to_tokens[idx]}','red', attrs=['bold']))